In [1]:
import pandas as pd
from math import sin, cos, sqrt, atan2, radians
import numpy as np

## Exercise 1: A/B Testing

We are a giant e-commerce company selling plants and pottery. We sell these items on our website as well as offer a 3-month subscription service where users get a new plant-pottery set every month of subscription with all required care instructions.
<br><br>
Suppose we are running a test comparing the current best model for pricing the items on our website and a new model we think might be better (the Challenger). We run the test showing these two different variants to our users but we realize there is an issue! The model is being shown to different numbers of people on different mobile devices (iOS and Android) and some of the users are also seeing a discount being offered. This makes the results of the test a bit hard to interpret. <br>

Answer the following questions about the experiment:<br>
1a. What is the probability that the Challenger is the superior model?<br>
1b. Based on your answer to number 1, would you be comfortable deciding yes/no on whether or not to change models?<br>
1c. If we decide to switch exclusively to the Challenger model for our iOS users, do we have a reasonable chance at getting 500 purchases in the first 10,000 views? what about 600?<br>

### Answer 1a:
**Assumption#1**: There was no bias in sampling users within Champion and Challenger model groups <br>

The outcome for each user's view is not continuous but binary: either "purchased item" or "not purchased item". A Chi-Square test is beneficial in such scenarios where we can compare the "purchase rates" of the two groups to decide if the challenger model is superior as compared to the champion. Hence, I calculated a new column called 'no_purchases'. <br>
Note: In the given data the purchase rate is as low as 5% for some cases. Lower rates can cause errors in estimating normal distributions. (Normal distribution is a valid approximation after 10-20 conversions). Hence, I chose Chi-Square. <br>
<br>For this experiment:

**Null Hypothesis**: There is no difference between the challenger and champion models' purchase rate (and the fact that we observed a difference is due to chance).
<br>
**Alternative Hypothesis**: The Challenger model has a higher purchase rate than the Champion.

In [2]:
models = pd.read_csv("ex2_table.csv")
models['no_purchases'] = models['total_views'] - models['number_of_purchases']
models

,device_type,variant,discount,total_views,number_of_purchases,no_purchases
0,android,Challenger,False,6010,189,5821
1,android,Challenger,True,331,16,315
2,android,Champion,False,1084,23,1061
3,android,Champion,True,54,3,51
4,iOS,Challenger,False,6905,336,6569
5,iOS,Challenger,True,1986,196,1790
6,iOS,Champion,False,6576,266,6310
7,iOS,Champion,True,2054,161,1893


Since during the experiment, the users were shown prices from models (champion vs challenger) on different mobile devices (iOS and Android) and with a presence or absence of discounts, these two factors will need to be controlled for while comparing the performance of both models. Controlling for device type and discount gives following 4 cases:<br><br>
CASE 1: device type = 'android' and discount = False <br>
CASE 2: device type = 'android' and discount = True <br>
CASE 3: device type = 'iOS' and discount = False <br>
CASE 4: device type = 'iOS' and discount = True <br>

Note: 
1. I will list the detailed steps for Case1. Calculations for the rest of the cases will follow similar logic. 

### CASE 1: device type = 'android' and discount = False

In [3]:
models[(models.discount==False) & (models.device_type=='android')]
# Following is the observed outcome table:

,device_type,variant,discount,total_views,number_of_purchases,no_purchases
0,android,Challenger,False,6010,189,5821
2,android,Champion,False,1084,23,1061


For our experiment,<br>
H₀: "the purchase rate is the same for the two models"<br>
H₁: "the purchase rate is higher for challenger model"<br>

**First step** is to model H₀ which says that both challenger and champion models' purchase rates follow the same Binomial Distribution. Hence, I will derive the theoretical purchase rate for H₀:
<br><br>
*Step 1:* <br>
p-hat = &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;((#challenger purchases) + (#champion purchases)) / <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;((#challenger total_views) + (#champion total_views)) <br><br>
p-hat = (189+23)/(6010+1084)<br>
p-hat = 0.029 <br><br>
*Step 2:*<br>
Multiply p-hat with total_views of both models to get theoretical purchase_rates. 
<br>Thus, under H₀, the theoretical outcome table is: <br>

| Variant  | total_views | purchases | no_purchases  |
|---|---|---|---|
| challenger | 6010 | 175 | 5835 |
| champion | 1084 | 31 | 1053 | <br>

**Second step** is to see how likely our observed samples are under H₀. For this, I will calculate D (squared relative distance between the theoretical and the observed distributions) and derive its corresponding p-value using Chi-square

In [4]:
from scipy.stats import chi2
import numpy as np

T_case1 = np.array([175, 31, 5835, 1053])
O_case1 = np.array([189, 23, 5821, 1061])

D_case1 = np.sum(np.square(T_case1-O_case1)/T_case1)

pvalue_case1 = chi2.sf(D_case1, df=1)

print("distance d: {0}\np-value: {1}".format(D_case1,pvalue_case1))

distance d: 3.278885259219726
p-value: 0.07017657889158617


*CONCLUSION:* P-value is 7%. With α criterion of 5%, we have pvalue>α and H₀ cannot be rejected.<br>Thus, we cannot conclude that challenger model outperforms champion for this case. In other words, with a confidence level of 95% we say that there is **No Significant Difference between the performance of two models.** 

### CASE 2 device type = 'android' and discount = True

In [5]:
models[(models.discount==True) & (models.device_type=='android')]
# Following is the observed outcome table:

,device_type,variant,discount,total_views,number_of_purchases,no_purchases
1,android,Challenger,True,331,16,315
3,android,Champion,True,54,3,51


**First step** I will derive the theoretical purchase rate for H₀:
<br><br>
*Step 1:* <br>
p-hat = (16+3)/(331+54)<br>
p-hat = 0.049 <br><br>
*Step 2:*<br>
Under H₀, the theoretical outcome table is: <br>

| Variant  | total_views | number_of_purchases | no_purchases  |
|---|---|---|---|
| challenger | 331 | 16 | 315 |
| champion | 54 | 3 | 51 | <br>

**Second step** is to see how likely our observed samples are under H₀. For this, I will calculate D (squared relative distance between the theoretical and the observed distributions) and derive its corresponding p-value using Chi-square

In [6]:
from scipy.stats import chi2
import numpy as np

T_case2 = np.array([16, 3, 315, 51])
O_case2 = np.array([16, 3, 315, 51])

D_case2 = np.sum(np.square(T_case2-O_case2)/T_case2)

pvalue_case2 = chi2.sf(D_case2, df=1)

print("distance d: {0}\np-value: {1}".format(D_case2,pvalue_case2))

distance d: 0.0
p-value: 1.0


*CONCLUSION:* P-value is 100%. With α criterion of 5%, we have pvalue>α and H₀ cannot be rejected. Even though the distance between theoretical and observed distributions is 0, p-value 100% suggests that the d=0 might be purely by chance. <br>Thus, we cannot conclude that challenger model outperforms champion for this case. In other words, with a confidence level of 95% we say that there is **No Significant Difference between the performance of two models.** 

### CASE 3 device type = 'iOS' and discount = False

In [7]:
models[(models.discount==False) & (models.device_type=='iOS')]
# Following is the observed outcome table:

,device_type,variant,discount,total_views,number_of_purchases,no_purchases
4,iOS,Challenger,False,6905,336,6569
6,iOS,Champion,False,6576,266,6310


**First step** I will derive the theoretical purchase rate for H₀:
<br><br>
*Step 1:* <br>
p-hat = (336+266)/(6905+6576)<br>
p-hat = 0.045 <br><br>
*Step 2:*<br>
Under H₀, the theoretical outcome table is: <br>

| Variant  | total_views | number_of_purchases | no_purchases  |
|---|---|---|---|
| challenger | 6905 | 311 | 6594 |
| champion | 6576 | 296 | 6280 | <br>

**Second step** is to see how likely our observed samples are under H₀. For this, I will calculate D (squared relative distance between the theoretical and the observed distributions) and derive its corresponding p-value using Chi-square

In [8]:
from scipy.stats import chi2
import numpy as np

T_case3 = np.array([311, 296, 6594, 6280])
O_case3 = np.array([336, 266, 6569, 6310])

D_case3 = np.sum(np.square(T_case3 - O_case3)/T_case3)

pvalue_case3 = chi2.sf(D_case3, df=1)

print("distance d: {0}\np-value: {1}".format(D_case3, pvalue_case3))

distance d: 5.2882820808865825
p-value: 0.021469388921317587


*CONCLUSION:* With α criterion of 5%, we have pvalue < α and H₀ can be rejected. <br>Thus, we conclude that with a confidence level of 95%, **the challenger model outperforms champion in this case.**

### CASE 4 device type = 'iOS' and discount = True

In [9]:
models[(models.discount==True) & (models.device_type=='iOS')]
# Following is the observed outcome table:

,device_type,variant,discount,total_views,number_of_purchases,no_purchases
5,iOS,Challenger,True,1986,196,1790
7,iOS,Champion,True,2054,161,1893


**First step** I will derive the theoretical purchase rate for H₀:
<br><br>
*Step 1:* <br>
p-hat = (196+161)/(1986+2054)<br>
p-hat = 0.088 <br><br>
*Step 2:*<br>
Under H₀, the theoretical outcome table is: <br>

| Variant  | total_views | number_of_purchases | no_purchases  |
|---|---|---|---|
| challenger | 1986 | 175 | 1811 |
| champion | 2054 | 181 | 1873 | <br>

**Second step** is to see how likely our observed samples are under H₀. For this, I will calculate D (squared relative distance between the theoretical and the observed distributions) and derive its corresponding p-value using Chi-square

In [10]:
from scipy.stats import chi2
import numpy as np

T_case4 = np.array([175, 181, 1811, 1873])
O_case4 = np.array([196, 161, 1790, 1893])

D_case4 = np.sum(np.square(T_case4 - O_case4)/T_case4)

pvalue_case4 = chi2.sf(D_case4, df=1)

print("distance d: {0}\np-value: {1}".format(D_case4, pvalue_case4))

distance d: 5.187017755149196
p-value: 0.022756233967973202


*CONCLUSION:* With α criterion of 5%, we have pvalue < α and H₀ can be rejected. <br>Thus, we conclude that with a confidence level of 95%, **the challenger model outperforms champion in this case.**

### Answer 1a.
To summarize: <br>
For device type = 'android' regardless of discount, there is not sufficient evidence to conclude that Challenger is the superiod model<br>
For device type = 'iOS' for both discount=True/False, there is roughly 98% probability that Challenger is the superior model.<br>

### Answer 1b. 
I would not be comfortable to change models to Challenger across all users (iOS + android devices)<br>

### Answer 1c.
**Assumption#1**: I will assess the chance of getting *atleast* 500 purchases in 10,000 view and *atleast* 600 purchases in 10,000 view<br>
It should be noted the iOS users who are offered discount are more likely to purchase an item as compared to the users who are not shown the discount. Hence, I will separately assess (for two groups: discount = True/False) whether we have a reasonable chance at getting 500 purchases in first 10,000 views and 600 in 10,000 views<br>

Steps - 
1. Calculate observed purchase rate from given data :: # of purchases/total_views
2. Calculate standard deviation from given data :: sqrt(p*(1-p)/n)
3. Calculate z-score of new purchases rates (500/10000 = 5% and 600/10000 = 6%)
4. Calculate probability for z-score
5. Conclude if we have reasonable chance
<br>
Alternatively, in a naive linear projection approach:
1. Multiply the observed 4.87% rate with 10,000 we get a projection of 487 purchases. 
2. Multiply the observed 9.87% rate with 10,000 we get a projection of 987 purchases. 
Hence, I will compare the 500 and 600 prize freeze goals with above projections. 
<br>

| Variant  | Discount | Observed purchases rate | Std. deviation  | New purchases |New purchases rate | Conclusion |
|---|---|---|---|---|---|---|
| challenger | False | 4.87% | 0.26% | 500 in 10,000 views | 5% | Z-score = 0.5. Probability of new purchase rate = 30%. Linear Projection: 487 purchases, falls short of the required 500. Thus, **we dont** have a reasonable chance |
| challenger | True | 9.87% | 0.48% | 500 in 10,000 views | 5% | Z-score = -10.145. Probability of new purchase rate = 100%. Linear Projection: 987 purchases, exceeds the required 500. Thus, **we have** a reasonable chance |
| challenger | False | 4.87% | 0.26% | 600 in 10,000 views | 6% | Z-score = 4.346. Probability of new purchase rate = 0.001%. Linear Projection: 487 purchases, falls short of the required 600. Thus, we **dont have** a reasonable chance  |
| challenger | True | 9.87% | 0.48% | 600 in 10,000 views | 6% | Z-score = -8.062. Probability of new purchase rate = 100%. Linear Projection: 987 purchases, exceeds the required 600. Thus, **we have** a reasonable chance |

## Exercise 2: Representation

We want to create a mathematical model of a user’s “purchase interaction” which can be described as a collection of searches. Our users are beginners who want to get into the planting game. Hence, they seldom make their purchases in the very first search. This requires us to represent this non-numeric search data such that we can draw quantitative conclusions.<br><br>
2a. How would you transform the collection of searches into a numeric vector representing a purchase?<br>
&nbsp;&nbsp;- Assume that we have hundreds of thousands of users and we want to represent all of their purchases this way.<br>
&nbsp;&nbsp;- We ideally want this to be a general representation we could use in multiple modeling projects, but we definitely care about finding similar purchases.<br>

**Note:** For confidentiality reasons, I will be not be able to show the data for this part of the exercise. Hence, I have listed the data definition in detail below.

**Data definition:**
<pre>
User ID                          Unique customer account Id
Purchase_Item                    Item the customer searched for
First Searched Date              Date of first search
Purchase Date                    Date of actual purchase
Purchase Type                    One-time purchase or Subscription 
</pre>

### Answer 2a:

| Variable  | Type | Description | Transformation  |
|---|---|---|---|
| purchase_id | New | Primary key representing each purchase | Generate unique numbers |
| user_id | Existing | Unique user id | N/A |
| purchase_type_one_time  | Derived | value = 1 when one time purchase else 0 | One-hot encode existing 'Purchase Type' column using pandas.get_dummies() |
| purchase_type_subscription | Derived | value = 1 when one way trip else 0  | One-hot encode existing 'Purchase Type' column using pandas.get_dummies() |
| item_category | Derived | Items represented as numbers | Label encode - give numerical value for each label |
| search_frequency | New | Number of times user searched for item | For each user, group by items & count records |
| first_searched_year | Derived | first search year | Transform using pandas datetime package: trips['first_searched_dt'].dt.year |
| first_searched_month | Derived | first search month | trips['first_searched_dt'].dt.month |
| first_searched_week | Derived | first search week | trips['first_searched_dt'].dt.week |
| first_searched_day | Derived | first search day of month | trips['first_searched_dt'].dt.day |
| first_searched_hour | Derived | first search hour | trips['first_searched_dt'].dt.hour |
| first_searched_dayofweek | Derived | first search day of week | trips['first_searched_dt'].dt.dayofweek |
| purchase_year | Derived | departure year | trips['departure_date'].dt.year |
| purchase_month | Derived | departure month | trips['departure_date'].dt.month |
| purchase_week | Derived | departure week | trips['departure_date'].dt.week |
| purchase_day | Derived | departure day of month | trips['departure_date'].dt.day |
| purchase_dayofweek | Derived | departure day of week | trips['departure_date'].dt.dayofweek |

<br>

*A lot of interesting analyses can be performed with above numerical vector and can be used for multiple modeling projects*:
1. purchase_window: Calculate difference between first_searched_dt and purchase_date
2. frequency of purchases being done in weekend vs weekdays
3. user's preference for searching/shopping (after work at a specific hour, at a specific day of the week)

## Exercise 3: Experiments and Data Collection

One of the core features of our website is that we advise our customers on whether to buy a subscription now or wait for the weather (temperature) to go down and purchase later. Our recommendations are critical for users because the types of plants we sell are very sensitive to the weather and will die within days if the weather conditions are not favourable. <br>
But what if our buy recommendation is wrong and the weather in fact worsens after the user subscribes to our service?  To lower the pain when this happens we refund users a certain % of their purchased amount if the weather worsens after they buy.
If you were the data scientist in charge of this project what information would you want to track to decide whether this recommendation feature is successful? What would you track to determine how to improve this feature?

### Answer 3:
This question can be thought of in terms of the following four scenarios. These scenarios are the outcome of buy/wait recommendation:<br>
shown_weather &#8594; buy &#8594; weather becomes pleasant : True positive &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(correctly recommended buy at best weather conditions) <br>
shown_weather &#8594; buy &#8594; weather worsens: **False Positive** &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(incorrectly recommended a positive action i.e. buy, when weather actually worsened)<br>
shown_weather &#8594; wait &#8594; weather worsense: True negative &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(correctly recommended wait and weather actually went worsened)<br>
shown_weather &#8594; wait &#8594; weather becomes pleasant: False negative &nbsp;&nbsp;&nbsp;(incorrectly recommended wait. Weather became pleasant instead of worse)
<br><br>
False Positive scenario above would trigger our refund action mentioned above. Success of this feature may mean multiple things. I would define it as follows:<br>
&nbsp;&nbsp;&nbsp;&nbsp;A. Suppose we have $1,000,000 budget for paying out all refunds in above scenarios. Then, all pay-outs should be maintained within budget<br>
&nbsp;&nbsp;&nbsp;&nbsp;B. Ensure customers do not churn 

For point#A, I would track below metrics: 
1. Total Type I error cost = sum of all refund payouts. Goal is to maintain this figure below $1,000,000
2. % of cases where refund in above false positive scenario was triggered = (# refunds)/(total # transactions)x100. Goal is to minimize this metric, indicating the frequency of correct recommendations and no customer hassle.
3. Standard deviation of amount of refund payout = STD(all refund payouts). Goal is to minimize this metric, indicating low variance in incorrect cost estimations. <br>

For point#B, I would track below metrics: <br>
1. Customer churn rate = (customers lost after refund interactions in a quarter)/(# customers at start of quarter). Note: the numerator will need to be controlled for other churn factors
2. If customer feedback/ratings are available, measure positive and negative sentiment rate.
<br>
Note: Additionally Type II error (false negative) might occur when wait recommendation is given but the weather becomes pleasant. To calculate cost of this error, I would look at the churn rate of customers waiting upto a certain time period after obeying the wait recommendation. But, not interacting with the app for further business.  
<br>

To improve this feature:
1. I would slowly and iteratively lower the % of the fare difference to be paid out by us and track customer sentiment and churn rate. Goal here is to minimize the amount to be paid out, ensuring we meet the budget while not losing customers. 
2. I would put a floor to the refund payout i.e. if the refund payout is greater than or equal to 50, only then a payout is eligible. 

## Exercise 4: Programming
Given the table of our warehouses and their locations (in latitude and longitude) below, write a function that takes a city code as input and returns the warehouses listed from nearest to furthest from the input city. This is for connecting user shipments from nearest warehouses when items are not available in the user's input city warehouse or when user is order from a city which does not have our warehouse. <br>
Use only the basic libraries (using sorting functions/methods provided by the standard library is definitely fine).

### Answer 4:
**Assumption#1**: Output should not return the input city itself as the first nearest element in the list <br>
**Assumption#2**: Nearest/farthest distances can be calculated using the shortest distance between two points on earth's surface using Haversine equation which results in 0.5% error <br>
**Assumption#3**: If  city code, other than the 8 codes listed in the dataframe below are provided as input, return error message <br>
**Assumption#4**: Given latitude, longitude are in degrees. Hence, convert to radians for calculating distance using Haversine formula in Python <br>
<br>*References:*<br>
Haversine formula - https://www.movable-type.co.uk/scripts/latlong.html

In [11]:
city = pd.read_csv("ex1_table.csv") 
city

,City Code,Lat,Long
0,CDG,49.012798,2.550000
1,CHC,-43.489399,172.531998
2,DYR,64.734901,177.740997
3,EWR,40.692501,-74.168701
4,HNL,21.318701,-157.921997
5,OME,64.512199,-165.445007
6,ONU,-20.650000,-178.699997
7,PEK,40.080101,116.584999


In [12]:
def distance(lat1, lon1, lat2, lon2):
    """ Function takes lat/lon coordinates for origin and destination airports and calculates distance between them
    
    Returns
    -------
    dist : float
    """
    # approximate radius of earth in miles
    R = 3958.8 
    # given latitudes and longitudes are in degrees, convert them to radians
    lat1 = radians(lat1) 
    lon1 = radians(lon1) 
    lat2 = radians(lat2)
    lon2 = radians(lon2)  
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    dist = R * c
    return dist

In [13]:
def warehouse_proximity(city_code):
    """ Function takes an airport code as input and returns the airports listed from nearest to furthest from the input

    Parameters
    ----------
    airport_code : str
    
    Returns
    -------
    airport_output : list
    """
    
    # check if airport code present in the dataframe read from file
    assert city_code in city['City Code'].values, "Sorry, input out of scope"
    
    # fetch input airport's location
    input_lat = city.loc[city['City Code'] == city_code]['Lat']
    input_lon = city.loc[city['City Code'] == city_code]['Long']
    
    # calculate distance of each airport from input and sort
    city['distance'] = city.apply(lambda row : distance(input_lat, input_lon, row['Lat'], row['Long']), axis = 1)
    city_output = city.sort_values(by='distance')
    
    # fetch the airport names from the sorted list and return
    city_output = city_output.iloc[1:,0].tolist()
    return city_output

# Test the function here:
output_df = warehouse_proximity('CDG')
print("Nearest to furthest warehouses from the input city are:")
print(*output_df, sep = "\n") 

Nearest to furthest warehouses from the input city are:
EWR
OME
DYR
PEK
HNL
ONU
CHC
